# CSS 201.5 - CSS MA Bootcamp

## Week 02 - Lecture 3 (morning)

# Data Wrangling

## Wrangling Categorical Variables

In [ ]:
# Loading packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import express as px
from plotly.subplots import make_subplots

# Dates and times handlers
from datetime import date
from datetime import timedelta
from datetime import datetime as dtm
from datetime import timezone as tmz

# Wrangling Dates and Times (cont'd)

In [ ]:
# Here are two dates
dts = ['5/17/2020', '12/2/2022']
dts2 = [date(2020, 5, 17), date(2022, 12, 2)]

In [ ]:
delta = dts2[1] - dts2[0]
delta.days

## Wrangling Dates and Times

### Timedeltas and duration

As we see before, we can create time differences (or timedeltas) by subtracting one date from another:

In [ ]:
begin = dtm(2021, 5, 5, 23, 20, 2)
end = dtm(2021, 7, 5, 8, 15, 22)
delta = end - begin

## Wrangling Dates and Times

### Timedeltas and duration

The handlers for time deltas and duration can be found in [here](https://images.datacamp.com/image/upload/v1666944896/Marketing/Blog/Working_with_Dates_and_Times_Cheat_Sheet.pdf)

In [ ]:
delta.total_seconds()

## Wrangling Dates and Times

**Exercise:** Read the following date and time correctly: "Jan 16, 2021 at 3:30 AM"

In [ ]:
# Your answers here

## Wrangling Dates and Times

### Timezone

In [ ]:
# Timezone PDT
PDT = tmz(timedelta(hours = -7))

# Date and time in an specific time zone
dtPDT = dtm(2021, 5, 12, 15, 23, 25, tzinfo = PDT)
print(dtPDT)

In [ ]:
# Or we can adjust
ET = tmz(timedelta(hours = -5))

# Before
print(dt1)

# After
dtET = dt1.astimezone(ET)
print(dtET)

## Wrangling Dates and Times

**Exercise:** Change `dt1` to India time zone (UTC+3:30).

In [ ]:
# Your answers here

## Wrangling Dates and Times

### Pandas

In [ ]:
# Datasets
dat2 = pd.read_csv('lakers.csv')

## Wrangling Dates and Times

**Exercise**: Explore this dataset.

In [ ]:
# Your code here

## Wrangling Dates and Times

### Pandas

To parse dates and times on pandas, we use the to_datetime method:

In [ ]:
# More on that later...

# Advanced Data Wrangling

## Roadmap

1. Drop variables

1. Sort variables

1. Indexing (basics)

1. Subsetting observations

1. Variable computations

1. Chaining

1. Stacking data

1. Joining data

1. Reshaping data

## Loading PErisk

In [2]:
perisk = pd.read_csv('PErisk.csv')
perisk.head(2)

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Argentina,0,-0.720775,1,3,9.690170
1,Australia,1,-6.907755,5,4,10.304840
2,Austria,1,-4.910337,5,4,10.100940
3,Bangladesh,0,0.775975,1,0,8.379768
4,Belgium,1,-4.617344,5,4,10.250120


In [2]:
tips = pd.read_csv('tips.csv')
tips.head(2)

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Argentina,0,-0.720775,1,3,9.690170
1,Australia,1,-6.907755,5,4,10.304840
2,Austria,1,-4.910337,5,4,10.100940
3,Bangladesh,0,0.775975,1,0,8.379768
4,Belgium,1,-4.617344,5,4,10.250120


## Dropping variables

This should be useful, especially when you have multiple variables.

In [3]:
new_perisk = perisk.drop(columns = ['courts', 'barb2', 'gdpw2'])

In [4]:
new_perisk.head(4)

,country,prsexp2,prscorr2
0,Argentina,1,3
1,Australia,5,4
2,Austria,5,4
3,Bangladesh,1,0


## Dropping variables

**Your turn**: Drop two variables of your choice in the `tips` dataset.

In [3]:
tips.head(2)
# Your answers here

## Changing position variables

Reindex helps you with this:

In [5]:
new_perisk = perisk.reindex(columns = ['country', 'courts', 'prsexp2', 'prscorr2', 'gdpw2', 'barb2'])

In [6]:
new_perisk.head(4)

,country,courts,prsexp2,prscorr2,gdpw2,barb2
0,Argentina,0,1,3,9.690170,-0.720775
1,Australia,1,5,4,10.304840,-6.907755
2,Austria,1,5,4,10.100940,-4.910337
3,Bangladesh,0,1,0,8.379768,0.775975


## Changing position variables

**Your turn**: Organize the `tips` dataset by placing the numeric variables first, and the other variables last.

In [3]:
tips.head(2)
# Your answers here

## Sorting variables

Useful for situations when need to check the dataset.

In [7]:
new_perisk = perisk.sort_values('gdpw2', ascending = True)

In [8]:
new_perisk.head(3)

,country,courts,barb2,prsexp2,prscorr2,gdpw2
33,Malawi,0,-1.469424,3,3,7.029973
13,Congo-Kinshasa,0,-2.323288,1,0,7.095064
8,Burma,0,1.604343,3,1,7.096721


In [9]:
new_perisk = perisk.sort_values('gdpw2', ascending = False)

In [10]:
new_perisk.head(3)

,country,courts,barb2,prsexp2,prscorr2,gdpw2
10,Canada,1,-6.907755,5,5,10.41018
51,Switzerland,1,-6.907755,5,5,10.34110
1,Australia,1,-6.907755,5,4,10.30484


## Sorting variables

**Your turn**: Sort the `tips` dataset by the value of the tip.

In [3]:
tips.head(2)
# Your answers here

## Indexing (basics)

Adding indexes:

In [11]:
new_perisk = perisk.set_index('country')

In [12]:
new_perisk.head(2)

,courts,barb2,prsexp2,prscorr2,gdpw2
country,,,,,
Argentina,0,-0.720775,1,3,9.69017
Australia,1,-6.907755,5,4,10.30484


Sort indexes:

In [13]:
new_perisk = new_perisk.sort_index(ascending = False)
new_perisk.head(2)

,courts,barb2,prsexp2,prscorr2,gdpw2
country,,,,,
Zimbabwe,0,-0.640321,3,2,7.965893
Zambia,0,0.965811,3,1,7.726213


Drop indexes:

In [14]:
new_perisk = new_perisk.reset_index()
new_perisk.head(2)

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Zimbabwe,0,-0.640321,3,2,7.965893
1,Zambia,0,0.965811,3,1,7.726213


## Indexing (basics)

**Your turn**: Set the `obs` as the index of the `tips` dataset. Then undo it.

In [3]:
tips.head(2)
# Your answers here

## Sampling

Sample a fraction of the data:

In [15]:
new_perisk = perisk.sample(frac = 0.05)
new_perisk

,country,courts,barb2,prsexp2,prscorr2,gdpw2
32,"Korea, South",0,-2.655795,4,1,9.422787
61,Zimbabwe,0,-0.640321,3,2,7.965893
46,Singapore,1,-4.848516,5,5,9.882724


Sample a given number of cases:

In [16]:
new_perisk = perisk.sample(n = 3)
new_perisk

,country,courts,barb2,prsexp2,prscorr2,gdpw2
3,Bangladesh,0,0.775975,1,0,8.379768
54,Togo,0,-4.229065,4,1,7.331715
17,Dominican Republic,0,-2.378862,2,2,8.899731


## Sampling

**Your turn**: Sample `20` observations from `tips`.

In [3]:
tips.head(2)
# Your answers here

## Subsetting

Subsetting using query:

In [17]:
new_perisk = perisk.query('gdpw2 > 10 and courts == 1')
new_perisk.head()

,country,courts,barb2,prsexp2,prscorr2,gdpw2
1,Australia,1,-6.907755,5,4,10.30484
2,Austria,1,-4.910337,5,4,10.10094
4,Belgium,1,-4.617344,5,4,10.25012
10,Canada,1,-6.907755,5,5,10.41018
16,Denmark,1,-6.907755,5,5,10.10651


## Subsetting

**Your turn**: Sample `50` observations from `tips`. Then, keep only the tips that are either bigger than or equal 10 dollars or came from a smoker. How many observations did you end up with?

In [3]:
tips.head(2)
# Your answers here

## Subsetting

Dropping duplicates:

In [18]:
new_perisk = perisk.sample(n = 5, replace = True, random_state = 479)
new_perisk

,country,courts,barb2,prsexp2,prscorr2,gdpw2
36,Morocco,0,-3.156958,3,1,8.780480
42,Philippines,0,-2.964776,1,1,8.384804
6,Botswana,1,-1.244868,4,3,8.777710
19,Finland,1,-6.907755,5,5,10.123670
36,Morocco,0,-3.156958,3,1,8.780480


In [19]:
new_perisk.drop_duplicates()

,country,courts,barb2,prsexp2,prscorr2,gdpw2
36,Morocco,0,-3.156958,3,1,8.780480
42,Philippines,0,-2.964776,1,1,8.384804
6,Botswana,1,-1.244868,4,3,8.777710
19,Finland,1,-6.907755,5,5,10.123670


## Subsetting

**Your turn**: A common operation in data science is called [`bootstrapping`](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)). It consists in randomly generate samples of the dataset you are working with replacement.

1. Create a sample with the `tips` with replacement, with the same size as the original data.

2. Check how many observations were repeated.

In [3]:
tips.head(2)
# Your answers here

## Variable operations

Creating new variables with computations (multiple columns):

In [20]:
new_perisk = perisk.assign(
    risk_expr = 5 - perisk.prsexp2,
    risk_corr = 5 - perisk.prscorr2,
)
new_perisk.head()

,country,courts,barb2,prsexp2,prscorr2,gdpw2,risk_expr,risk_corr
0,Argentina,0,-0.720775,1,3,9.690170,4,2
1,Australia,1,-6.907755,5,4,10.304840,0,1
2,Austria,1,-4.910337,5,4,10.100940,0,1
3,Bangladesh,0,0.775975,1,0,8.379768,4,5
4,Belgium,1,-4.617344,5,4,10.250120,0,1


## Variable operations

**Your turn**: Create a variable `share_bill_tip`, that computes the fraction of the bill that was give as a `tip`.

In [3]:
tips.head(2)
# Your answers here

## Variable operations

Quantiles of a variable:

In [21]:
perisk.gdpw2.quantile(q = [0, 0.25, 0.5, 0.75, 1])

0.00     7.029973
0.25     8.381027
0.50     9.185412
0.75     9.889280
1.00    10.410180
Name: gdpw2, dtype: float64

And we can bin by quantiles:

In [22]:
new_perisk = perisk.assign(
    gdpw2_bin = pd.qcut(perisk.gdpw2, q = 4)
)
new_perisk.head()

,country,courts,barb2,prsexp2,prscorr2,gdpw2,gdpw2_bin
0,Argentina,0,-0.720775,1,3,9.690170,"(9.185, 9.889]"
1,Australia,1,-6.907755,5,4,10.304840,"(9.889, 10.41]"
2,Austria,1,-4.910337,5,4,10.100940,"(9.889, 10.41]"
3,Bangladesh,0,0.775975,1,0,8.379768,"(7.029, 8.381]"
4,Belgium,1,-4.617344,5,4,10.250120,"(9.889, 10.41]"


## Quantile cuts

**Your turn**: Cut the `share_bill_tip` into three categories. Then build a table of this variable.

In [3]:
tips.head(2)
# Your answers here

## Variable operations

Little bit of a 0-1 index:

In [23]:
new_perisk = perisk.assign(
    z1_barb2 = (perisk.barb2 - perisk.barb2.min()) / (perisk.barb2.max() - perisk.barb2.min())
)
new_perisk.head()

,country,courts,barb2,prsexp2,prscorr2,gdpw2,z1_barb2
0,Argentina,0,-0.720775,1,3,9.690170,0.669211
1,Australia,1,-6.907755,5,4,10.304840,0.000000
2,Austria,1,-4.910337,5,4,10.100940,0.216050
3,Bangladesh,0,0.775975,1,0,8.379768,0.831107
4,Belgium,1,-4.617344,5,4,10.250120,0.247741


## Zero-One Indexing

**Your turn**: Create a variable `zero_one_totbill`, that transforms the total bill into a zero - one variable.

In [3]:
tips.head(2)
# Your answers here

## Variable operations

Standardizing values or taking absolute values:

In [24]:
stdz = lambda x: (x - x.mean()) / x.std()
new_perisk = perisk.assign(
    stdz_barb2 = stdz(perisk.barb2),
    stdz_gdpw2 = stdz(perisk.gdpw2),
    abs_barb2 = perisk.barb2.abs()
)
new_perisk.head()

,country,courts,barb2,prsexp2,prscorr2,gdpw2,stdz_barb2,stdz_gdpw2,abs_barb2
0,Argentina,0,-0.720775,1,3,9.690170,0.814411,0.668164,0.720775
1,Australia,1,-6.907755,5,4,10.304840,-1.470960,1.301672,6.907755
2,Austria,1,-4.910337,5,4,10.100940,-0.733146,1.091523,4.910337
3,Bangladesh,0,0.775975,1,0,8.379768,1.367286,-0.682399,0.775975
4,Belgium,1,-4.617344,5,4,10.250120,-0.624919,1.245275,4.617344


Clipping values: force lower and higher to be of a given value (danger zone!).

In [25]:
new_perisk = new_perisk.assign(
    stdz_barb2 = new_perisk.stdz_barb2.clip(lower = -1, upper = 1),
    stdz_gdpw2 = new_perisk.stdz_gdpw2.clip(lower = -1, upper = 1),
)
new_perisk.head(3)

,country,courts,barb2,prsexp2,prscorr2,gdpw2,stdz_barb2,stdz_gdpw2,abs_barb2
0,Argentina,0,-0.720775,1,3,9.69017,0.814411,0.668164,0.720775
1,Australia,1,-6.907755,5,4,10.30484,-1.000000,1.000000,6.907755
2,Austria,1,-4.910337,5,4,10.10094,-0.733146,1.000000,4.910337


## Standardizing

**Your turn**: 

1. Standardize the total bill.
2. Create a new variable that standardize the tips, clipping values to be between -2 and 2 standard deviations.
3. Count the values within and outside these bounds.

Do you know what they mean?

In [3]:
tips.head(2)
# Your answers here

## Chaining

This is useful when you want to run multiple commands at once.

In [26]:
new_perisk = (perisk.assign(expr_risk = 5 - perisk.prsexp2,
                            corr_risk = 5 - perisk.prsexp2)
                    .query('courts == 1')
                    .sample(n = 5)
                    .set_index('country')
                    .sort_index()
             )

In [27]:
new_perisk.head()

,courts,barb2,prsexp2,prscorr2,gdpw2,expr_risk,corr_risk
country,,,,,,,
Italy,1,-6.907755,4,3,10.260780,1,1
New Zealand,1,-6.907755,5,5,10.176260,0,0
Portugal,1,-2.459625,4,3,9.444543,1,1
Singapore,1,-4.848516,5,5,9.882724,0,0
Switzerland,1,-6.907755,5,5,10.341100,0,0


## Stacking data

Suppose you have two datasets, both with half of the data you need, and the same variables in both datasets.

For instance:

In [28]:
# First dataset
perisk_1sthalf = perisk.loc[0:1]
perisk_1sthalf

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Argentina,0,-0.720775,1,3,9.69017
1,Australia,1,-6.907755,5,4,10.30484


In [29]:
# Second dataset
perisk_2ndhalf = perisk.loc[2:3]
perisk_2ndhalf

,country,courts,barb2,prsexp2,prscorr2,gdpw2
2,Austria,1,-4.910337,5,4,10.100940
3,Bangladesh,0,0.775975,1,0,8.379768


## Stacking data

To stack the data, you should do:

In [30]:
pd.concat([perisk_1sthalf, perisk_2ndhalf])

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Argentina,0,-0.720775,1,3,9.690170
1,Australia,1,-6.907755,5,4,10.304840
2,Austria,1,-4.910337,5,4,10.100940
3,Bangladesh,0,0.775975,1,0,8.379768


## Stacking data

But what if the variables are in different order, with possibly some differences from one dataset to the other?

In [31]:
perisk_1sthalf = (
    perisk.loc[0:1]
          .drop(columns = ['barb2'])
          .reindex(columns = ['country', 'prscorr2', 'gdpw2', 'courts', 'prsexp2'])
)
perisk_1sthalf

,country,prscorr2,gdpw2,courts,prsexp2
0,Argentina,3,9.69017,0,1
1,Australia,4,10.30484,1,5


In [32]:
perisk_2ndhalf = (
    perisk.loc[2:3]
          .drop(columns = ['gdpw2'])
          .reindex(columns = ['country', 'courts', 'prsexp2', 'prscorr2', 'barb2'])
)
perisk_2ndhalf

,country,courts,prsexp2,prscorr2,barb2
2,Austria,1,5,4,-4.910337
3,Bangladesh,0,1,0,0.775975


In [33]:
pd.concat([perisk_1sthalf, perisk_2ndhalf])

,country,prscorr2,gdpw2,courts,prsexp2,barb2
0,Argentina,3,9.69017,0,1,NaN
1,Australia,4,10.30484,1,5,NaN
2,Austria,4,NaN,1,5,-4.910337
3,Bangladesh,0,NaN,0,1,0.775975


## Join Data

Suppose you have two datasets that have a common key, with different types of information in them. How to we join them together?

In [34]:
perisk_inc1 = (
    perisk.loc[0:4]
          .drop(columns = ['prsexp2', 'prscorr2', 'gdpw2'])
)
perisk_inc1

,country,courts,barb2
0,Argentina,0,-0.720775
1,Australia,1,-6.907755
2,Austria,1,-4.910337
3,Bangladesh,0,0.775975
4,Belgium,1,-4.617344


In [35]:
perisk_inc2 = (
    perisk.loc[1:5]
          .drop(columns = ['courts', 'barb2'])
)
perisk_inc2

,country,prsexp2,prscorr2,gdpw2
1,Australia,5,4,10.304840
2,Austria,5,4,10.100940
3,Bangladesh,1,0,8.379768
4,Belgium,5,4,10.250120
5,Bolivia,0,0,8.583543


## Join Data

Inner joins:

In [36]:
pd.merge(perisk_inc1, perisk_inc2, how = 'inner', on = 'country')

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Australia,1,-6.907755,5,4,10.304840
1,Austria,1,-4.910337,5,4,10.100940
2,Bangladesh,0,0.775975,1,0,8.379768
3,Belgium,1,-4.617344,5,4,10.250120


## Join Data

Left joins:

In [37]:
pd.merge(perisk_inc1, perisk_inc2, how = 'left', on = 'country')

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Argentina,0,-0.720775,NaN,NaN,NaN
1,Australia,1,-6.907755,5.0,4.0,10.304840
2,Austria,1,-4.910337,5.0,4.0,10.100940
3,Bangladesh,0,0.775975,1.0,0.0,8.379768
4,Belgium,1,-4.617344,5.0,4.0,10.250120


## Join Data

Right joins:

In [38]:
pd.merge(perisk_inc1, perisk_inc2, how = 'right', on = 'country')

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Australia,1.0,-6.907755,5,4,10.304840
1,Austria,1.0,-4.910337,5,4,10.100940
2,Bangladesh,0.0,0.775975,1,0,8.379768
3,Belgium,1.0,-4.617344,5,4,10.250120
4,Bolivia,NaN,NaN,0,0,8.583543


## Join Data

Full (outer) joins:

In [39]:
pd.merge(perisk_inc1, perisk_inc2, how = 'outer', on = 'country')

,country,courts,barb2,prsexp2,prscorr2,gdpw2
0,Argentina,0.0,-0.720775,NaN,NaN,NaN
1,Australia,1.0,-6.907755,5.0,4.0,10.304840
2,Austria,1.0,-4.910337,5.0,4.0,10.100940
3,Bangladesh,0.0,0.775975,1.0,0.0,8.379768
4,Belgium,1.0,-4.617344,5.0,4.0,10.250120
5,Bolivia,NaN,NaN,0.0,0.0,8.583543


## Join Data

Diagnostics one: matched?

In [40]:
perisk_inc1[perisk_inc1.country.isin(perisk_inc2.country)].country

1     Australia
2       Austria
3    Bangladesh
4       Belgium
Name: country, dtype: object

Diagnostics two: Unmatched?

In [41]:
perisk_inc1[~perisk_inc1.country.isin(perisk_inc2.country)].country

0    Argentina
Name: country, dtype: object

## Join Data

Now reversing:

Diagnostics one: matched?

In [42]:
perisk_inc2[perisk_inc2.country.isin(perisk_inc1.country)].country

1     Australia
2       Austria
3    Bangladesh
4       Belgium
Name: country, dtype: object

Diagnostics two: Unmatched?

In [43]:
perisk_inc2[~perisk_inc2.country.isin(perisk_inc1.country)].country

5    Bolivia
Name: country, dtype: object

In [44]:
# cases data
cases = pd.DataFrame({
  'country': ["Afghanistan", "Brazil", "China"],
  1999: [745, 37737, 212258],
  2000: [2666, 80488, 213766]  
})
cases

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


## Reshaping data (gather)

Suppose you have this data:

In [45]:
cases_spread = pd.DataFrame({
  'country': ["Afghanistan", "Brazil", "China"],
  1999: [745, 37737, 212258],
  2000: [2666, 80488, 213766]  
})
cases_spread

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


The first thing we can do in here is to `gather` this data:

In [46]:
cases_new = pd.melt(cases_spread, id_vars = 'country', var_name = 'year', value_name = 'cases')
cases_new

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


## Reshaping data (spread)

Now, suppose you have this data:

In [47]:
cases_new

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


But you want to go back to the previous pattern:

In [48]:
pd.pivot(cases_new, index = 'country', columns = 'year', values = 'cases').reset_index()

/var/folders/n1/8dmszvzd4wd4235wc5m18xjc0000gn/T/ipykernel_5546/637181755.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pd.pivot(cases_new, index = 'country', columns = 'year', values = 'cases').reset_index()


year,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


# Great work!